# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [250]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, count, when, isnull, collect_list
from pyspark.sql.types import *
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import functions as f
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [19]:
import pandas as pd
%matplotlib inline
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [3]:
# create a Spark session
sc =  SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [224]:
filename = 'medium-sparkify-event-data.json.gz'
df = sc.read.json(filename)
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [225]:
def show_df(df):
    """
        ARGS:
            - RDD
        RETURNS:
            - Prints dataframe
    """
    return pd.DataFrame(df.take(5),columns=df.columns).head()

In [233]:
def df_dimentions(df):
    """
        Prints number of Rows and Columns
        ARGS:
            - Dataframe
    """
    print(f"row count: {df.count()}, column count: {len(df.columns)}")
df_dimentions(df)

row count: 543705, column count: 25


In [227]:
## remove special characters from column
def remove_special_chars(df):
    """
        removes special characters from data
        ARGS:
            - Dataframe
        RETURNS:
            - clear Dataframe
    """
    for field in df.schema.fields:
        if field.dataType == StringType():
            df = df.withColumn(field.name,regexp_replace(field.name,'[^a-zA-Z0-9\,\-]',''))
    
    return df
df = remove_special_chars(df)
show_df(df)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,MartinOrford,LoggedIn,Joseph,M,20,Morales,597.55057,free,"CorpusChristi,TX",PUT,NextSong,1532063507000,292,GrandDesigns,200,1538352011000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985125Safari53736",293
1,JohnBrownsBody,LoggedIn,Sawyer,M,74,Larson,380.21179,free,"Houston-TheWoodlands-SugarLand,TX",PUT,NextSong,1538069638000,97,Bulls,200,1538352025000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985143Safari53736",98
2,Afroman,LoggedIn,Maverick,M,184,Santiago,202.37016,paid,"Orlando-Kissimmee-Sanford,FL",PUT,NextSong,1535953455000,178,BecauseIGotHigh,200,1538352118000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985143Safari53736",179
3,None,LoggedIn,Maverick,M,185,Santiago,NaN,paid,"Orlando-Kissimmee-Sanford,FL",PUT,Logout,1535953455000,178,None,307,1538352119000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985143Safari53736",179
4,LilyAllen,LoggedIn,Gianna,F,22,Campos,194.53342,paid,"Mobile,AL",PUT,NextSong,1535931018000,245,SmileRadioEdit,200,1538352124000,Mozilla50WindowsNT61WOW64rv310Gecko20100101Firefox310,246


In [228]:
def convert_unixtimestamp(df,ts_col):
    """
        converts unix timestamp to date time
        ARGS:
            - df: Dataframe with Unixtimestamp
            - ts: unixtimestamp column name
        RETURNS:
            - dataframe with new datetime column
    """
    df = df.withColumn("dateTime",from_unixtime(col(ts_col).cast(LongType())/1000).cast(TimestampType()))
    df = df.withColumn("month",month(col('dateTime')))
    df = df.withColumn("date",from_unixtime(col(ts_col)/1000).cast(DateType()))
    df = df.withColumn("location",split(col('location'),',').getItem(1))
    df = df.filter(col('userId').isNotNull())
    return df


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,dateTime,month,date
0,MartinOrford,LoggedIn,Joseph,M,20,Morales,597.55057,free,TX,PUT,NextSong,1532063507000,292,GrandDesigns,200,1538352011000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985125Safari53736",293,2018-10-01 05:30:11,10,2018-10-01
1,JohnBrownsBody,LoggedIn,Sawyer,M,74,Larson,380.21179,free,TX,PUT,NextSong,1538069638000,97,Bulls,200,1538352025000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985143Safari53736",98,2018-10-01 05:30:25,10,2018-10-01
2,Afroman,LoggedIn,Maverick,M,184,Santiago,202.37016,paid,FL,PUT,NextSong,1535953455000,178,BecauseIGotHigh,200,1538352118000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985143Safari53736",179,2018-10-01 05:31:58,10,2018-10-01
3,None,LoggedIn,Maverick,M,185,Santiago,NaN,paid,FL,PUT,Logout,1535953455000,178,None,307,1538352119000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985143Safari53736",179,2018-10-01 05:31:59,10,2018-10-01
4,LilyAllen,LoggedIn,Gianna,F,22,Campos,194.53342,paid,AL,PUT,NextSong,1535931018000,245,SmileRadioEdit,200,1538352124000,Mozilla50WindowsNT61WOW64rv310Gecko20100101Firefox310,246,2018-10-01 05:32:04,10,2018-10-01


In [230]:
#temp_df = df.withColumn('regDatetime',from_unixtime(col('registration')/1000).cast(DateType()))
def calculate_active_duration(df):
    """
        Calculates number of days a user is active since the day he registerd
        ARGS:
            - dataframe
        REURNS:
            - Dataframe with new colum activeDuration
    """
    df = df.withColumn('regDatetime',from_unixtime(col('registration')/1000).cast(DateType()))
    user_duration_df = df.groupBy('UserId','regDatetime').agg(max('date').alias('lastInterationDate')) 
    diff  = (f.unix_timestamp('lastInterationDate')-f.unix_timestamp('regDatetime'))
    duration_df = user_duration_df.withColumn('activeDuration',diff/86400)
    
    df = df.join(duration_df,on='UserId')
    return df                              

,userId,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,dateTime,month,date,regDatetime,regDatetime,lastInterationDate,activeDuration
0,293,MartinOrford,LoggedIn,Joseph,M,20,Morales,597.55057,free,TX,PUT,NextSong,1532063507000,292,GrandDesigns,200,1538352011000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985125Safari53736",2018-10-01 05:30:11,10,2018-10-01,2018-07-20,2018-07-20,2018-11-22,125.0
1,98,JohnBrownsBody,LoggedIn,Sawyer,M,74,Larson,380.21179,free,TX,PUT,NextSong,1538069638000,97,Bulls,200,1538352025000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985143Safari53736",2018-10-01 05:30:25,10,2018-10-01,2018-09-27,2018-09-27,2018-11-30,64.0
2,179,Afroman,LoggedIn,Maverick,M,184,Santiago,202.37016,paid,FL,PUT,NextSong,1535953455000,178,BecauseIGotHigh,200,1538352118000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985143Safari53736",2018-10-01 05:31:58,10,2018-10-01,2018-09-03,2018-09-03,2018-12-01,89.0
3,179,None,LoggedIn,Maverick,M,185,Santiago,NaN,paid,FL,PUT,Logout,1535953455000,178,None,307,1538352119000,"Mozilla50MacintoshIntelMacOSX1094AppleWebKit53736KHTML,likeGeckoChrome3601985143Safari53736",2018-10-01 05:31:59,10,2018-10-01,2018-09-03,2018-09-03,2018-12-01,89.0
4,246,LilyAllen,LoggedIn,Gianna,F,22,Campos,194.53342,paid,AL,PUT,NextSong,1535931018000,245,SmileRadioEdit,200,1538352124000,Mozilla50WindowsNT61WOW64rv310Gecko20100101Firefox310,2018-10-01 05:32:04,10,2018-10-01,2018-09-03,2018-09-03,2018-11-05,63.0


In [275]:
# Auth type
df.select("Auth").distinct().collect()

[Row(Auth='Cancelled'),
 Row(Auth='Guest'),
 Row(Auth='LoggedIn'),
 Row(Auth='LoggedOut')]

In [282]:
# Types of pages
df.select('page').distinct().collect()

[Row(page='Cancel'),
 Row(page='CancellationConfirmation'),
 Row(page='Home'),
 Row(page='ThumbsDown'),
 Row(page='Downgrade'),
 Row(page='ThumbsUp'),
 Row(page='SubmitDowngrade'),
 Row(page='Logout'),
 Row(page='About'),
 Row(page='Settings'),
 Row(page='Login'),
 Row(page='Register'),
 Row(page='SaveSettings'),
 Row(page='NextSong'),
 Row(page='RollAdvert'),
 Row(page='AddtoPlaylist'),
 Row(page='Help'),
 Row(page='Upgrade'),
 Row(page='SubmitUpgrade'),
 Row(page='Error'),
 Row(page='SubmitRegistration'),
 Row(page='AddFriend')]

In [314]:
def create_label(df):
    """
        Create label columns containing 1,0 for cancellation page and auth is cancled
        ARGS:
            - df: Dataframe 
        RETURNS:
            - Dataframe with labeled columns
    """
    temp_label_df = df.withColumn('label', when((col('page').isin(['Cancellation Confirmation','Cancel'])) | \
                               (col('auth') == 'Cancelled'),0).otherwise(1)) \
                                .groupby('userId').agg(sum('label').alias('label')) \
                                .withColumn('label',when(col('label') >=1,1).otherwise(0))
    df = df.join(temp_label_df,on='userId')
    return df
show_df(create_label(df))

,userId,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,dateTime,month,date,regDatetime,regDatetime,lastInterationDate,activeDuration,label
0,100010,Darude,LoggedIn,Darianna,F,0,Carpenter,226.08934,free,CT,PUT,NextSong,1538016340000,62,Sandstorm,200,1538991392000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 15:06:32,10,2018-10-08,2018-09-27,2018-09-27,2018-10-11,14.0,1
1,100010,Justice,LoggedIn,Darianna,F,1,Carpenter,285.41342,free,CT,PUT,NextSong,1538016340000,62,PhantomPart15AlbumVersion,200,1538991618000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 15:10:18,10,2018-10-08,2018-09-27,2018-09-27,2018-10-11,14.0,1
2,100010,FiveIronFrenzy,LoggedIn,Darianna,F,2,Carpenter,236.09424,free,CT,PUT,NextSong,1538016340000,62,Canada,200,1538991903000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 15:15:03,10,2018-10-08,2018-09-27,2018-09-27,2018-10-11,14.0,1
3,100010,Carpenters,LoggedIn,Darianna,F,3,Carpenter,250.25261,free,CT,PUT,NextSong,1538016340000,62,TicketToRide,200,1538992139000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 15:18:59,10,2018-10-08,2018-09-27,2018-09-27,2018-10-11,14.0,1
4,100010,Uffie,LoggedIn,Darianna,F,4,Carpenter,227.52608,free,CT,PUT,NextSong,1538016340000,62,DismissedUffie,200,1538992389000,"Mozilla50iPhoneCPUiPhoneOS712likeMacOSXAppleWebKit537512KHTML,likeGeckoVersion70Mobile11D257Safari953753",2018-10-08 15:23:09,10,2018-10-08,2018-09-27,2018-09-27,2018-10-11,14.0,1


In [338]:
def user_session_sum():
    """
        Create a column containing the sum of ItemInSession column
        RETURNS:
            - Dataframe containing column sumItemInSession 
    """
    item_sum_df = df.groupby('userId').agg(sum(col('itemInSession')).alias('sumItemInSession'))
    df =  df.join(item_sum_df,on='userId')
    return df

In [ ]:
def count_missings(spark_df,sort=True):
    """
        Counts number of nulls and nans in each column
        ARGS:
            - spark_df: RDD dataframe 
    """
    df = spark_df.select([f.count(f.when(f.isnan(c) | f.isnull(c), c)).alias(c) for (c,c_type) in spark_df.dtypes if c_type not in ('timestamp', 'string', 'date')]).toPandas()

    if len(df) == 0:
        print("There are no any missing values!")
        return None

    if sort:
        return df.rename(index={0: 'missing_value_count'}).T.sort_values("missing_value_count",ascending=False)

    return df


In [340]:
cleaned_df = df.toPandas()
plt.figure(figsize=(15,5))
sns.barplot(x='itemInSession',data=cleaned_df)
plt.show()

TypeError: cannot label index with a null key

<Figure size 1080x360 with 0 Axes>

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [ ]:
count_missings(df)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [ ]:
df = convert_unixtimestamp(df,'ts')
df = calculate_active_duration(df)
show_df(df)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.